<a href="https://colab.research.google.com/github/vishenka-git/churn_prediction/blob/main/Churn_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Анализ сообщений, отправленных пользователям**



In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
pip install pymorphy2

     |████████████████████████████████| 61kB 1.9MB/s 
     |████████████████████████████████| 8.2MB 7.0MB/s 


In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
import spacy
import string
import re
import pymorphy2
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

%matplotlib inline

In [5]:
#Настройки для удобства чтения сообщений

pd.options.display.max_colwidth = 1000
pd.options.display.max_rows = 1000

In [6]:
#Вывод данных

df_actions = pd.read_csv('/content/gdrive/My Drive/messages/actions.csv')
df_messages = pd.read_csv('/content/gdrive/My Drive/messages/messages.csv')

In [7]:
df_actions.head(10)

,id,subject,body,type
0,239,NaN,NaN,push
1,77,NaN,NaN,push
2,213,Бесплатно доставим вам первый заказ👍,"Оформляйте скорее, доставим в удобное время!",push
3,218,👍 Новые скидки METRO в Instamart,Успей заказать продукты по выгодной цене!,push
4,221,Дарим 500 рублей за приглашенного друга!,"Обнови приложение, получи промокод и делись им с друзьями",push
5,230,Скидка {{event.product.discount_percent}}% на ваш любимый товар,{{event.product.name}} из магазина {{event.store.name}},push
6,238,👀 Новые скидки METRO в Instamart,Успей заказать продукты по выгодной цене!,push
7,259,👌Обновились скидки METRO в Instamart,Заказывайте акционные товары пока они в наличии!,push
8,302,🏝 Скоро в отпуск и пригодится скидка!,50% скидка на чемоданы American Tourister в Instamart,push
9,306,👌Обновились скидки METRO в Instamart,Заказывайте акционные товары пока они в наличии!,push


In [8]:
df_actions.iloc[:, 1:3]

,subject,body
0,NaN,NaN
1,NaN,NaN
2,Бесплатно доставим вам первый заказ👍,"Оформляйте скорее, доставим в удобное время!"
3,👍 Новые скидки METRO в Instamart,Успей заказать продукты по выгодной цене!
4,Дарим 500 рублей за приглашенного друга!,"Обнови приложение, получи промокод и делись им с друзьями"
5,Скидка {{event.product.discount_percent}}% на ваш любимый товар,{{event.product.name}} из магазина {{event.store.name}}
6,👀 Новые скидки METRO в Instamart,Успей заказать продукты по выгодной цене!
7,👌Обновились скидки METRO в Instamart,Заказывайте акционные товары пока они в наличии!
8,🏝 Скоро в отпуск и пригодится скидка!,50% скидка на чемоданы American Tourister в Instamart
9,👌Обновились скидки METRO в Instamart,Заказывайте акционные товары пока они в наличии!


In [9]:
df_actions['type'].value_counts()

push     577
email    270
sms      115
Name: type, dtype: int64

In [10]:
df_actions.columns

Index(['id', 'subject', 'body', 'type'], dtype='object')

In [13]:
df_messages.head(10)

,sent,action_id,user_id
0,1.588847e+09,1512.0,1775692
1,1.588847e+09,1512.0,1775687
2,1.588847e+09,1512.0,1775684
3,1.588847e+09,1512.0,1775678
4,1.588847e+09,1512.0,1775660
5,1.588847e+09,1512.0,1775657
6,1.588847e+09,1512.0,1775656
7,1.588847e+09,1512.0,1775654
8,1.588847e+09,1512.0,1775647
9,1.588847e+09,1512.0,1775638


In [14]:
df_messages.dtypes

sent         float64
action_id    float64
user_id        int64
dtype: object

In [15]:
df_messages['sent'] = pd.to_datetime(df_messages['sent'], unit='s')

In [16]:
df_messages.dtypes

sent         datetime64[ns]
action_id           float64
user_id               int64
dtype: object

In [17]:
df_messages['month'] = pd.DatetimeIndex(df_messages['sent']).month

In [18]:
df_messages.head(10)

,sent,action_id,user_id,month
0,2020-05-07 10:26:54,1512.0,1775692,5
1,2020-05-07 10:26:44,1512.0,1775687,5
2,2020-05-07 10:26:32,1512.0,1775684,5
3,2020-05-07 10:25:56,1512.0,1775678,5
4,2020-05-07 10:24:20,1512.0,1775660,5
5,2020-05-07 10:23:53,1512.0,1775657,5
6,2020-05-07 10:23:48,1512.0,1775656,5
7,2020-05-07 10:23:40,1512.0,1775654,5
8,2020-05-07 10:23:19,1512.0,1775647,5
9,2020-05-07 10:22:56,1512.0,1775638,5


In [19]:
df_messages['action_id'] = df_messages.action_id.astype(int)

In [20]:
df_messages.dtypes

sent         datetime64[ns]
action_id             int64
user_id               int64
month                 int64
dtype: object

In [ ]:
# df_messages['message_date'] = [d.date() for d in df_messages['sent']]
# df_messages['message_time'] = [d.time() for d in df_messages['sent']]

In [22]:
df_messages.isnull().sum()

sent         0
action_id    0
user_id      0
month        0
dtype: int64

In [23]:
df_actions.isnull().sum()

id           0
subject    439
body       430
type         0
dtype: int64

In [24]:
df_actions[df_actions.isnull().any(axis=1)]

,id,subject,body,type
0,239,NaN,NaN,push
1,77,NaN,NaN,push
12,342,NaN,NaN,push
17,2043,NaN,NaN,push
18,2029,NaN,NaN,push
50,2015,NaN,NaN,push
61,1132,NaN,NaN,push
66,1154,NaN,NaN,push
73,1183,NaN,NaN,push
74,1186,NaN,NaN,push


In [25]:
df_subj = df_actions[["subject"]]
df_actions["subject"] = df_actions["subject"].astype(str)

In [26]:
df_body = df_actions[["body"]]
df_actions["body"] = df_actions["body"].astype(str)

In [27]:
df_subj.head()

,subject
0,NaN
1,NaN
2,Бесплатно доставим вам первый заказ👍
3,👍 Новые скидки METRO в Instamart
4,Дарим 500 рублей за приглашенного друга!


In [28]:
df_subj["text_lower"] = df_subj["subject"].str.lower().replace("ё", "е")
df_subj.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,subject,text_lower
0,NaN,NaN
1,NaN,NaN
2,Бесплатно доставим вам первый заказ👍,бесплатно доставим вам первый заказ👍
3,👍 Новые скидки METRO в Instamart,👍 новые скидки metro в instamart
4,Дарим 500 рублей за приглашенного друга!,дарим 500 рублей за приглашенного друга!


In [29]:
df_subj['text_lower']= df_subj['text_lower'].fillna('пусто')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
df_subj.head()

,subject,text_lower
0,NaN,пусто
1,NaN,пусто
2,Бесплатно доставим вам первый заказ👍,бесплатно доставим вам первый заказ👍
3,👍 Новые скидки METRO в Instamart,👍 новые скидки metro в instamart
4,Дарим 500 рублей за приглашенного друга!,дарим 500 рублей за приглашенного друга!


In [31]:
df_subj["text_wo_punct"] = df_subj["text_lower"].apply(lambda text: re.sub('[^a-zA-Zа-яА-Я0-9]+', ' ', text))
df_subj.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,subject,text_lower,text_wo_punct
0,NaN,пусто,пусто
1,NaN,пусто,пусто
2,Бесплатно доставим вам первый заказ👍,бесплатно доставим вам первый заказ👍,бесплатно доставим вам первый заказ
3,👍 Новые скидки METRO в Instamart,👍 новые скидки metro в instamart,новые скидки metro в instamart
4,Дарим 500 рублей за приглашенного друга!,дарим 500 рублей за приглашенного друга!,дарим 500 рублей за приглашенного друга


In [32]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
", ".join(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


'и, в, во, не, что, он, на, я, с, со, как, а, то, все, она, так, его, но, да, ты, к, у, же, вы, за, бы, по, только, ее, мне, было, вот, от, меня, еще, нет, о, из, ему, теперь, когда, даже, ну, вдруг, ли, если, уже, или, ни, быть, был, него, до, вас, нибудь, опять, уж, вам, ведь, там, потом, себя, ничего, ей, может, они, тут, где, есть, надо, ней, для, мы, тебя, их, чем, была, сам, чтоб, без, будто, чего, раз, тоже, себе, под, будет, ж, тогда, кто, этот, того, потому, этого, какой, совсем, ним, здесь, этом, один, почти, мой, тем, чтобы, нее, сейчас, были, куда, зачем, всех, никогда, можно, при, наконец, два, об, другой, хоть, после, над, больше, тот, через, эти, нас, про, всего, них, какая, много, разве, три, эту, моя, впрочем, хорошо, свою, этой, перед, иногда, лучше, чуть, том, нельзя, такой, им, более, всегда, конечно, всю, между'

In [33]:
STOPWORDS = set(stopwords.words('russian'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df_subj["text_wo_stop"] = df_subj["text_wo_punct"].apply(lambda text: remove_stopwords(text))
df_subj.head()

,subject,text_lower,text_wo_punct,text_wo_stop
0,NaN,пусто,пусто,пусто
1,NaN,пусто,пусто,пусто
2,Бесплатно доставим вам первый заказ👍,бесплатно доставим вам первый заказ👍,бесплатно доставим вам первый заказ,бесплатно доставим первый заказ
3,👍 Новые скидки METRO в Instamart,👍 новые скидки metro в instamart,новые скидки metro в instamart,новые скидки metro instamart
4,Дарим 500 рублей за приглашенного друга!,дарим 500 рублей за приглашенного друга!,дарим 500 рублей за приглашенного друга,дарим 500 рублей приглашенного друга


In [34]:
morph = MorphAnalyzer()

In [35]:
df_subj["text_lemmatized"] = df_subj["text_wo_stop"].apply(lambda text: morph.parse(text)[0].normal_form)
df_subj.head(20)

,subject,text_lower,text_wo_punct,text_wo_stop,text_lemmatized
0,NaN,пусто,пусто,пусто,пусто
1,NaN,пусто,пусто,пусто,пусто
2,Бесплатно доставим вам первый заказ👍,бесплатно доставим вам первый заказ👍,бесплатно доставим вам первый заказ,бесплатно доставим первый заказ,бесплатно доставим первый заказ
3,👍 Новые скидки METRO в Instamart,👍 новые скидки metro в instamart,новые скидки metro в instamart,новые скидки metro instamart,новые скидки metro instamart
4,Дарим 500 рублей за приглашенного друга!,дарим 500 рублей за приглашенного друга!,дарим 500 рублей за приглашенного друга,дарим 500 рублей приглашенного друга,дарим 500 рублей приглашенного друг
5,Скидка {{event.product.discount_percent}}% на ваш любимый товар,скидка {{event.product.discount_percent}}% на ваш любимый товар,скидка event product discount percent на ваш любимый товар,скидка event product discount percent ваш любимый товар,скидка event product discount percent ваш любимый товар
6,👀 Новые скидки METRO в Instamart,👀 новые скидки metro в instamart,новые скидки metro в instamart,новые скидки metro instamart,новые скидки metro instamart
7,👌Обновились скидки METRO в Instamart,👌обновились скидки metro в instamart,обновились скидки metro в instamart,обновились скидки metro instamart,обновились скидки metro instamart
8,🏝 Скоро в отпуск и пригодится скидка!,🏝 скоро в отпуск и пригодится скидка!,скоро в отпуск и пригодится скидка,скоро отпуск пригодится скидка,скоро отпуск пригодится скидка
9,👌Обновились скидки METRO в Instamart,👌обновились скидки metro в instamart,обновились скидки metro в instamart,обновились скидки metro instamart,обновились скидки metro instamart


In [36]:
df_subj.drop(['subject'], axis=1, inplace=True)
df_subj.drop(["text_lower"], axis=1, inplace=True)
df_subj.drop(["text_wo_punct"], axis=1, inplace=True)
df_subj.drop(["text_wo_stop"], axis=1, inplace=True)

In [37]:
df_subj.head()

,text_lemmatized
0,пусто
1,пусто
2,бесплатно доставим первый заказ
3,новые скидки metro instamart
4,дарим 500 рублей приглашенного друг


In [59]:
from collections import Counter
cnt = Counter()
for text in df_subj['text_lemmatized'].values:
    cnt[text] += 1
        
cnt.most_common(35)

[('пусто', 439),
 ('заказывать стало проща', 78),
 ('скидка', 17),
 ('спасибо выбор сбермаркет', 12),
 ('промокод заказ instamart', 12),
 ('ой ваш заказ', 10),
 ('встречайте сбермаркет новое название instamart', 10),
 ('вашей корзине остались товар', 7),
 ('скидка обмен ваше мнение', 7),
 ('ваши 250 рублей сгорят скоро нужный', 7),
 ('дарим самое важный', 6),
 ('урра начислены бонус', 6),
 ('обновились скидки metro instamart', 5),
 ('доставляем ашать', 5),
 ('экономьте делая заказ', 5),
 ('завтра сгорит ваша скидка 200 рублей instamart', 5),
 ('сбермаркет открыт заходите очень ждать', 5),
 ('успели получить скидка', 4),
 ('подарок обмен ваше мнение', 4),
 ('бесплатная доставка обмен ваше мнение', 4),
 ('упускаете бесплатную доставку обмен ваше мнение', 4),
 ('использовали бесплатную доставку скоро закончиться', 4),
 ('сегодня готовим брауни m m s приступить', 4),
 ('бесплатно доставим первый заказ', 3),
 ('новые скидки metro instamart', 3),
 ('скидка 99 рублей заказ instamart', 3),
 ('

In [38]:
df_subj['Frequency']= df_subj.groupby('text_lemmatized')['text_lemmatized'].transform('size')

In [39]:
df_h_actions = pd.concat([df_actions, df_subj], axis=1)

In [40]:
df_h_actions.head()

,id,subject,body,type,text_lemmatized,Frequency
0,239,nan,nan,push,пусто,439
1,77,nan,nan,push,пусто,439
2,213,Бесплатно доставим вам первый заказ👍,"Оформляйте скорее, доставим в удобное время!",push,бесплатно доставим первый заказ,3
3,218,👍 Новые скидки METRO в Instamart,Успей заказать продукты по выгодной цене!,push,новые скидки metro instamart,3
4,221,Дарим 500 рублей за приглашенного друга!,"Обнови приложение, получи промокод и делись им с друзьями",push,дарим 500 рублей приглашенного друг,1


In [41]:
df_h_actions.drop(df_h_actions.columns[1:5], axis=1, inplace=True)

In [43]:
from sklearn.preprocessing import LabelEncoder  
number = LabelEncoder()
df_h_actions['Frequency'] = number.fit_transform(df_h_actions['Frequency'].astype('str'))

In [44]:
df_h_actions.head()

,id,Frequency
0,239,7
1,77,7
2,213,5
3,218,5
4,221,0


In [45]:
df_h_actions.rename(columns ={'id': "action_id", 'Frequency': "groupno"}, inplace=True)

In [47]:
df_h_actions.head()

,action_id,groupno
0,239,7
1,77,7
2,213,5
3,218,5
4,221,0


In [48]:
df_fin = df_messages.merge(df_h_actions, on= 'action_id', how='left')

In [49]:
df_fin.head()

,sent,action_id,user_id,month,groupno
0,2020-05-07 10:26:54,1512,1775692,5,6
1,2020-05-07 10:26:44,1512,1775687,5,6
2,2020-05-07 10:26:32,1512,1775684,5,6
3,2020-05-07 10:25:56,1512,1775678,5,6
4,2020-05-07 10:24:20,1512,1775660,5,6


In [50]:
df_fin.drop(df_fin.columns[0:2], axis=1, inplace=True)

In [53]:
df_fin.head()

,user_id,month,groupno
0,1775692,5,6
1,1775687,5,6
2,1775684,5,6
3,1775678,5,6
4,1775660,5,6


In [ ]:
# df_fin.to_csv('/content/gdrive/My Drive/messages/result.csv')

In [54]:
df_fin.shape

(18275907, 3)

In [55]:
 df_fin['key'] = df_fin['user_id'].astype(str) + "_" + df_fin['month'].astype(str)

In [56]:
df_fin.head()

,user_id,month,groupno,key
0,1775692,5,6,1775692_5
1,1775687,5,6,1775687_5
2,1775684,5,6,1775684_5
3,1775678,5,6,1775678_5
4,1775660,5,6,1775660_5


In [57]:
df_fin.drop(df_fin.columns[0:2], axis=1, inplace=True)

In [58]:
df_fin.head()

,groupno,key
0,6,1775692_5
1,6,1775687_5
2,6,1775684_5
3,6,1775678_5
4,6,1775660_5


In [ ]:
df_fin.to_csv('/content/gdrive/My Drive/messages/messages_group_wkey.csv')